# Kalman Filter

## Imports

In [1]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from tqdm import tqdm

## Load all csv names

In [10]:
f = open("./data/data_names.txt", "r")
file_names = f.readlines()
f.close()

csv_names = []
for f_n in file_names:
    csv_names.append(f_n[:-1])

csv_names[:34] #[34:]

['raspberry_02-Feb-2022-19-57_aceleración repentina_Data-Muy3Rwan4A9AhGjSY_j.csv',
 'raspberry_02-Feb-2022-19-58_aceleración repentina_Data-Mv1lsdW8oLkQkP2XcI9.csv',
 'raspberry_02-Feb-2022-19-59_aceleración repentina_Data-Mv1lxNTPcnt0_t3C724.csv',
 'raspberry_02-Feb-2022-20-00_aceleración repentina_Data-Mv1m25ILfD2v20bDi5Q.csv',
 'raspberry_02-Feb-2022-20-02_frenada repentina_Data-Mv2E_1BnfkXOP6gRM1J.csv',
 'raspberry_02-Feb-2022-20-04_frenada repentina_Data-Mv2Fjh23ZUv0zs8puNO.csv',
 'raspberry_02-Feb-2022-20-05_frenada repentina_Data-Mv2GE5kuLbtayOKAADh.csv',
 'raspberry_02-Feb-2022-20-07_frenada repentina_Data-Mv2GjLnIwNMl56-k3B4.csv',
 'raspberry_02-Feb-2022-20-08_giro derecho agresivo_Data-Mv2HSOB7ABwlXlcqhJ8.csv',
 'raspberry_02-Feb-2022-20-09_giro derecho agresivo_Data-Mv2ISfQUC6sTST7r5vK.csv',
 'raspberry_02-Feb-2022-20-11_giro derecho agresivo_Data-Mv2J5sXkAQlaBieponq.csv',
 'raspberry_02-Feb-2022-20-14_giro izquierdo agresivo_Data-Mv2JJRm4Wr7hGezVRa4.csv',
 'raspberry_02-Feb

## Plot functions

In [5]:
variables_dict = {'speed': 'Velocidad', 'latitude':'Lat', 'longitude':'lng', 'accPosition': 'Presión del acelerador',
                  'accX': 'Aceleración en X', 'accY': 'Aceleración en Y', 'accZ': 'Aceleración en Z',
                  'magX': 'Fuerza magnética en X', 'magY': 'Fuerza magnética en Y', 'magZ': 'Fuerza magnética en Z',
                  'velAngX': 'Velocidad angular en X', 'velAngY': 'Velocidad angular en Y', 'velAngZ': 'Velocidad angular en Z'}
units_dict = {'speed': 'm/s', 'latitude':'°', 'longitude':'°', 'accPosition': '% de presión',
              'accX': 'm/s\u00B2', 'accY': 'm/s\u00B2', 'accZ': 'm/s\u00B2',
              'magX': '\u03BC T', 'magY': '\u03BC T', 'magZ': '\u03BC T',
              'velAngX': 'rad/seg', 'velAngY': 'rad/seg', 'velAngZ': 'rad/seg'}


def update_layout(fig, chart_title: str, **kwargs):
    """Update layout for every chart

    Args:
        fig: A plotly figure
        chart_title (str): A title for chart

    Returns:
        fig: the updated layout for the plotly figure
    """
    # Get kwards data if is needed
    x_title = kwargs.get('xaxis_title', None)
    y_title = kwargs.get('yaxis_title', None)
    height_size = kwargs.get('height', 450)

    # Update layout of the figure
    fig.update_layout(
        title={'text': chart_title, 'x': 0.5},
        legend_title='Tipo de evento',
        xaxis_title=x_title,
        yaxis_title=y_title,
        template='plotly_white',
        autosize=True,
        height=height_size,
        font=dict(
            family="BlinkMacSystemFont,-apple-system,Segoe UI,Roboto,Oxygen,Ubuntu, \
                    Cantarell,Fira Sans,Droid Sans,Helvetica Neue,Helvetica,Arial,sans-serif",
            size=14,
            color="#363636"
        )
    )

    return fig


def line_chart(df, Y_measured ,Y_hat):
    """Line chart created with plotly graphic object

    Args:
        df (DataFrame): The dataframe that contains the data to plot
        variable (str): The specific variable to plot

    Returns:
        fig: A plotly figure
    """
    x = df['timestamp']
    variable = Y_measured.name
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x,
        y=Y_measured,
        name='Measured data',
        marker_color='#00D1B1',
        text=df["id"],
        hovertemplate="X = %{x}<br>Y = %{y}<br>ID = %{text}"
    ))
    fig.add_trace(go.Scatter(
        x=x,
        y=Y_hat,
        name='Filter data',
        marker_color='#3273dc',
        text=df["id"],
        hovertemplate="X = %{x}<br>Y = %{y}<br>ID = %{text}"
    ))

    fig = update_layout(fig, f'Gráfico de linea de la {variables_dict[variable]}',
                        xaxis_title='Marca de Tiempo', yaxis_title=units_dict[variable])
    fig.update_yaxes(nticks=12)
    fig.update_xaxes(nticks=12, tickangle=45)

    return fig

## Using filter

In [3]:
from pykalman import KalmanFilter

variables = ["accX", "accY", "accZ", "velAngX", "velAngY", "velAngZ", "magX", "magY", "magZ"]
for name in ["raspberry_02-Feb-2022-19-57_aceleración repentina_Data-Muy3Rwan4A9AhGjSY_j.csv"]:
    df = pd.read_csv(f"./data/{name}")
    for var in tqdm(variables, desc=name):
        data = df[var]

        kf = KalmanFilter(initial_state_mean = data.iloc[0], n_dim_obs=1)
        filter_data = kf.em(data).filter(data)[0].T[0]
        filter_data_s = pd.Series(np.array(filter_data), name=var)
        df[var] = filter_data_s

        #fig = line_chart(df, data, new_data_s)
        #fig.show()

    df.to_csv(f"./filtered_data/{name}", index=False)

raspberry_02-Feb-2022-19-57_aceleración repentina_Data-Muy3Rwan4A9AhGjSY_j.csv: 100%|██████████| 9/9 [00:41<00:00,  4.60s/it]


## Plot filter data

In [6]:
df = pd.read_csv("./data/raspberry_02-Feb-2022-19-57_aceleración repentina_Data-Muy3Rwan4A9AhGjSY_j.csv")
df_filter = pd.read_csv("./filtered_data/raspberry_02-Feb-2022-19-57_aceleración repentina_Data-Muy3Rwan4A9AhGjSY_j.csv")
for var in variables:
    data = df[var]
    filter_data = df_filter[var]
    fig = line_chart(df, data, filter_data)
    fig.show()

In [90]:
from sklearn.metrics import mean_squared_error

max_standby = 7250
min_standby = 6680
var_noise = "accZ"


g_noise = df.loc[(df['id'] <= max_standby) & (df["id"] >=min_standby), var_noise]
g_noise

y_true = np.array([10]*len(g_noise))

mean_squared_error(y_true, g_noise)

1.2758263365434335

In [144]:
import time
my_df = pd.DataFrame([[1,2],[1,2]], columns=["a","b"])
print(my_df)
for i in ["HELLO", "MOTO"]:
    for z in tqdm(range(3), desc=i):
        n_d = pd.Series([5,5])
        my_df["a"] = n_d
        x = 1
        time.sleep(1)

print(my_df)

   a  b
0  1  2
1  1  2


MOTO: 100%|██████████| 3/3 [00:03<00:00,  1.00s/it]

   a  b
0  5  2
1  5  2
